In [155]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import utils as helper
import torch
from dataset import TimeSeriesDataset
from torch.utils.data import DataLoader

In [156]:
test_data = pd.read_csv('dataset/BA_weather_6-23_jan.csv', parse_dates=['Date'])
mean, std = np.load('dataset/stat.npy')
test_data = test_data.iloc[::-1]
test_data.reset_index(drop=True, inplace=True)
test_data["next_T"] = test_data["T"].shift(1)

In [157]:
test_data

,Date,T,P0,U,Ff,next_T
0,2023-06-01 00:00:00,9,753.9,71,4,NaN
1,2023-06-01 00:30:00,9,753.9,71,4,9.0
2,2023-06-01 01:00:00,9,753.9,71,4,9.0
3,2023-06-01 01:30:00,8,754.7,76,4,9.0
4,2023-06-01 02:00:00,8,754.7,76,3,8.0
...,...,...,...,...,...,...
857,2023-01-23 21:30:00,2,759.6,87,3,1.0
858,2023-01-23 22:00:00,1,759.6,100,3,2.0
859,2023-01-23 22:30:00,1,760.4,93,3,1.0
860,2023-01-23 23:00:00,2,760.4,87,2,1.0


In [158]:
X, y = helper.process_data(test_data)

/home/maco/Documents/git-repos/WeatherProphet/data_utils.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["sin_day_of_year"] = np.sin(
/home/maco/Documents/git-repos/WeatherProphet/data_utils.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["cos_day_of_year"] = np.cos(
/home/maco/Documents/git-repos/WeatherProphet/data_utils.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [159]:
X

,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,P0,T,U,Ff
1,0.501242,-0.865307,0.258819,0.965926,753.9,9,71,4
3,0.501242,-0.865307,0.500000,0.866025,754.7,8,76,3
5,0.501242,-0.865307,0.707107,0.707107,754.7,9,71,3
7,0.501242,-0.865307,0.866025,0.500000,754.7,8,76,3
9,0.501242,-0.865307,0.965926,0.258819,755.4,8,76,4
...,...,...,...,...,...,...,...,...
851,0.385663,0.922640,-0.965926,0.258819,759.7,3,87,2
853,0.385663,0.922640,-0.866025,0.500000,759.7,3,87,2
855,0.385663,0.922640,-0.707107,0.707107,759.6,1,93,3
857,0.385663,0.922640,-0.500000,0.866025,759.6,1,100,3


In [160]:
y

1      9.0
3      8.0
5      9.0
7      8.0
9      9.0
      ... 
851    3.0
853    3.0
855    2.0
857    2.0
859    1.0
Name: next_T, Length: 430, dtype: float64

In [161]:
X, mean, std = helper.normalize_data(X)

In [162]:
X, Y = X.to_numpy(), y.to_numpy()

In [163]:
dataset = TimeSeriesDataset(X, y, 1)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [164]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load('WP200-mlp_w1_sch60_shuffle_ds.pt')
model.to(device)
model.eval()

ClassicWeatherProphet(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [165]:

a = []
for i, batch in enumerate(dataloader):
    x = batch[0].type(torch.FloatTensor).to(device)
    y = batch[1].type(torch.FloatTensor).to(device).reshape(-1, 1)
    print(x)
    print(y)
    y_pred = model(x)
    a.append(y_pred.detach().cpu().numpy())
    break

tensor([[[ 1.0131, -1.8482,  0.3640,  1.3675,  0.8248,  1.6455, -1.1111,
           0.0798]]], device='cuda:0')
tensor([[9.]], device='cuda:0')


In [166]:
a

[array([[26.792059]], dtype=float32)]

In [167]:
X[0]

array([ 1.01305681, -1.84824608,  0.36396058,  1.36751848,  0.82476945,
        1.64548205, -1.11109484,  0.07977525])

In [168]:
Y[0:10]

array([9., 8., 9., 8., 9., 8., 7., 6., 6., 9.])